In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from IPython.display import SVG
import IPython

import keras
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model, model_to_dot
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop

import lieb_liniger_state as lls

np.random.seed(7)

# Necessary since otherwise we can't compare the accuracy between naive and ML initial guesses.
keras.backend.set_floatx('float64')

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
no_of_particles = 5
no_of_states = 20000

In [ ]:
ground_state = lls.lieb_liniger_state(1, no_of_particles, no_of_particles)
Is = np.zeros((no_of_states, no_of_particles))
lambdas = np.zeros((no_of_states, no_of_particles))
for i in range(no_of_states):
    if i % 1000 == 0:
        print(f"Generated {i}/{no_of_states} states")
    bethe_numbers = lls.generate_bethe_numbers(no_of_particles, ground_state.Is)
    llstate = lls.lieb_liniger_state(1, no_of_particles, no_of_particles, bethe_numbers)
    llstate.lambdas = 2 * np.pi / llstate.L * llstate.Is
    no_of_iterations = llstate.calculate_rapidities_newton()
    Is[i] = bethe_numbers
    lambdas[i] = llstate.lambdas

In [ ]:
def neural_net(no_of_particles):
    model = Sequential()
    model.add(Dense(units=no_of_particles, kernel_initializer='lecun_uniform', activation='tanh', input_dim=no_of_particles))
#     model.add(Dense(units=no_of_particles**2, kernel_initializer='lecun_uniform', activation='tanh'))
    model.add(Dense(units=no_of_particles**3, kernel_initializer='lecun_uniform', activation='tanh'))
#     model.add(Dense(units=no_of_particles**2, kernel_initializer='lecun_uniform', activation='tanh'))
    model.add(Dense(units=no_of_particles, kernel_initializer='lecun_uniform'))
    model.compile(loss='mse', optimizer=RMSprop())
    model.summary()
    return model

In [ ]:
epochs=100
model = neural_net(no_of_particles)
keras.utils.plot_model(model, to_file='test_keras_plot_model.png', show_shapes=True)
# IPython.display.Image('test_keras_plot_model.png')

history = model.fit(x=Is, y=lambdas, epochs=epochs, verbose=1, validation_split=0.05, 
                    callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0002, patience=5, verbose=1, mode='auto')]
                   )

In [ ]:
plt.plot(range(1, len(history.history["val_loss"]) + 1), history.history["val_loss"])
plt.xlabel("Epoch")
plt.ylabel("Validation error")
plt.show()

print(model.predict(Is[0].reshape(1, -1), batch_size=1)[0])
print(Is[0])
print(lambdas[0])

In [ ]:
nstates = 1000

dp = pd.DataFrame(data=np.zeros(shape=(6000, 3)), columns=["Iterations", "Method", "Damped"])

for k in range(nstates):
    bethe_numbers = lls.generate_bethe_numbers(no_of_particles, ground_state.Is)

    # Naive Bethe (damped)
    llstate = lls.lieb_liniger_state(1, no_of_particles, no_of_particles, bethe_numbers)
    llstate.lambdas = 2 * np.pi / llstate.L * llstate.Is
    no_of_iterations = llstate.calculate_rapidities_newton(enable_damping=True, printing=False)
    dp.iloc[k] = [no_of_iterations, "Naive Bethe", True]
    
    # Naive Bethe (undamped)
    llstate = lls.lieb_liniger_state(1, no_of_particles, no_of_particles, bethe_numbers)
    llstate.lambdas = 2 * np.pi / llstate.L * llstate.Is
    no_of_iterations = llstate.calculate_rapidities_newton(enable_damping=False, printing=False)
    dp.iloc[k+nstates] = [no_of_iterations, "Naive Bethe", False]

    # Machine learning (damped)
    llstate = lls.lieb_liniger_state(1, no_of_particles, no_of_particles, bethe_numbers)
    llstate.lambdas = model.predict(bethe_numbers.reshape(1, -1), batch_size=1)[0]
    no_of_iterations = llstate.calculate_rapidities_newton(enable_damping=True, printing=False)
    dp.iloc[k+2*nstates] = [no_of_iterations, "ML", True]
    
    # Machine learning (undamped)
    llstate = lls.lieb_liniger_state(1, no_of_particles, no_of_particles, bethe_numbers)
    llstate.lambdas = model.predict(bethe_numbers.reshape(1, -1), batch_size=1)[0]
    no_of_iterations = llstate.calculate_rapidities_newton(enable_damping=False, printing=False)
    dp.iloc[k+3*nstates] = [no_of_iterations, "ML", False]


    # Random (damped)
    llstate = lls.lieb_liniger_state(1, no_of_particles, no_of_particles, bethe_numbers)
    llstate.lambdas = np.random.normal(0, no_of_particles, no_of_particles)
    no_of_iterations = llstate.calculate_rapidities_newton(enable_damping=True, printing=False)
    dp.iloc[k+4*nstates] = [no_of_iterations, "Random", True]
    
    # Random (undamped)
    llstate = lls.lieb_liniger_state(1, no_of_particles, no_of_particles, bethe_numbers)
    llstate.lambdas = np.random.normal(0, no_of_particles, no_of_particles)
    no_of_iterations = llstate.calculate_rapidities_newton(enable_damping=False, printing=False)
    dp.iloc[k+5*nstates] = [no_of_iterations, "Random", False]

In [ ]:
sns.set_style("ticks")
ax = sns.barplot(x="Method", y="Iterations", hue="Damped", data=dp, ci="sd")
sns.despine()
       
print(dp.loc[(dp['Method'] == "Naive Bethe") & dp['Damped']]["Iterations"].mean())
print(dp.loc[(dp['Method'] == "Naive Bethe") & dp['Damped']]["Iterations"].std())

print(dp.loc[(dp['Method'] == "Naive Bethe") & ~dp['Damped']]["Iterations"].mean())
print(dp.loc[(dp['Method'] == "Naive Bethe") & ~dp['Damped']]["Iterations"].std(), "\n")

print(dp.loc[(dp['Method'] == "ML") & dp['Damped']]["Iterations"].mean())
print(dp.loc[(dp['Method'] == "ML") & dp['Damped']]["Iterations"].std())

print(dp.loc[(dp['Method'] == "ML") & ~dp['Damped']]["Iterations"].mean())
print(dp.loc[(dp['Method'] == "ML") & ~dp['Damped']]["Iterations"].std(), "\n")

print(dp.loc[(dp['Method'] == "Random") & dp['Damped']]["Iterations"].mean())
print(dp.loc[(dp['Method'] == "Random") & dp['Damped']]["Iterations"].std())

print(dp.loc[(dp['Method'] == "Random") & ~dp['Damped']]["Iterations"].mean())
print(dp.loc[(dp['Method'] == "Random") & ~dp['Damped']]["Iterations"].std())

In [ ]:
# times = pd.DataFrame(data=np.zeros(shape=(30, 3)), columns=["Times", "Method", "Damped"])

# # Naive Bethe (undamped)
# for t in range(10):
#     start_time = time.time()
#     for k in range(1000):
#         bethe_numbers = lls.generate_bethe_numbers(no_of_particles, ground_state.Is)

#         llstate = lls.lieb_liniger_state(1, no_of_particles, no_of_particles, bethe_numbers)
#         llstate.lambdas = 2 * np.pi / llstate.L * llstate.Is
#         no_of_iterations = llstate.calculate_rapidities_newton(enable_damping=False, printing=False)
#     times.iloc[t] = [time.time() - start_time, "Naive Bethe", "False"]

# # Machine learning (undamped)
# for t in range(10):
#     start_time = time.time()
#     for k in range(1000):
#         bethe_numbers = lls.generate_bethe_numbers(no_of_particles, ground_state.Is)

#         llstate = lls.lieb_liniger_state(1, no_of_particles, no_of_particles, bethe_numbers)
#         llstate.lambdas = model.predict(bethe_numbers.reshape(1, -1), batch_size=1)[0]
#         no_of_iterations = llstate.calculate_rapidities_newton(enable_damping=False, printing=False)
#     times.iloc[t+10] = [time.time() - start_time, "ML", "False"]
    
# # Random (undamped)
# for t in range(10):
#     start_time = time.time()
#     for k in range(1000):
#         bethe_numbers = lls.generate_bethe_numbers(no_of_particles, ground_state.Is)

#         llstate = lls.lieb_liniger_state(1, no_of_particles, no_of_particles, bethe_numbers)
#         llstate.lambdas = np.random.normal(0, no_of_particles, no_of_particles)
#         no_of_iterations = llstate.calculate_rapidities_newton(enable_damping=False, printing=False)
#     times.iloc[t+20] = [time.time() - start_time, "Random", "False"]

In [ ]:
# sns.set_style("ticks")
# ax = sns.barplot(x="Method", y="Times", data=times, ci="sd")
# sns.despine()

# print(times.loc[(times['Method'] == "ML")]["Times"].mean())
# print(times.loc[(times['Method'] == "ML")]["Times"].std(), "\n")

# print(times.loc[(times['Method'] == "Naive Bethe")]["Times"].mean())
# print(times.loc[(times['Method'] == "Naive Bethe")]["Times"].std(), "\n")

# print(times.loc[(times['Method'] == "Random")]["Times"].mean())
# print(times.loc[(times['Method'] == "Random")]["Times"].std())